In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [2]:
# department_links=['https://khi.nu.edu.pk/faculty-php/',
#                  'https://khi.nu.edu.pk/department-of-cyber-security/',
#                  'https://khi.nu.edu.pk/department-of-artificial-intelligence/',
#                  'https://khi.nu.edu.pk/department-of-software-engineering/',
#                  'https://khi.nu.edu.pk/department-of-electrical-engineering/',
#                  'https://khi.nu.edu.pk/department-of-management-sciences/',
#                  'https://khi.nu.edu.pk/department-of-sciences-humanities/']
# print(len(department_links))

In [27]:
url = 'https://khi.nu.edu.pk/'
r = requests.get(url)
soup = BeautifulSoup(r.content, 'html.parser')
# Find all anchor elements with an 'href' attribute
anchor_tags = soup.find_all('a', href=True)

# Initialize a set to store unique links
department_links = set()

for tag in anchor_tags:
    href = tag['href']
    if href.startswith('https://khi.nu.edu.pk/department-of') or href.startswith('https://khi.nu.edu.pk/faculty-php/'):
        department_links.add(href)
        
department_links = list(department_links)

department_links

['https://khi.nu.edu.pk/department-of-sciences-humanities/',
 'https://khi.nu.edu.pk/department-of-electrical-engineering/',
 'https://khi.nu.edu.pk/department-of-management-sciences/',
 'https://khi.nu.edu.pk/department-of-cyber-security/',
 'https://khi.nu.edu.pk/department-of-artificial-intelligence/',
 'https://khi.nu.edu.pk/faculty-php/',
 'https://khi.nu.edu.pk/department-of-software-engineering/']

# Iterate for each Department

In [30]:
id_=[]
name=[]
designation=[]
phdSupervisor=[]
email=[]
dept=[]
ext=[]
img=[]
highesteducation=[]

In [31]:
for i in range(len(department_links)):
    response = requests.get(department_links[i])
    soup = BeautifulSoup(response.content, 'html.parser')
    f = soup.findAll('div', class_='gdlr-core-personnel-list gdlr-core-outer-frame-element clearfix')
    
    for j in range(len(f)):
        faculty=f[j]
        
        ################################
        facultyName = faculty.find('h3', class_='gdlr-core-personnel-list-title').text.strip().split(',')[0]
        facultyDesignation = faculty.find('div', class_='gdlr-core-personnel-list-position gdlr-core-info-font gdlr-core-skin-caption').text.strip()
        facultyEmail = faculty.find('div', class_='kingster-personnel-info-list kingster-type-email').text.strip()
        facultyDepartment = faculty.find_previous('h1', class_='kingster-page-title').text.strip()
        facultyExtension = faculty.find('div', class_='kingster-personnel-info-list kingster-type-phone').text.strip()
        
        facultyLink = faculty.find('h3', class_='gdlr-core-personnel-list-title').find_next('a')['href']
        
        re = requests.get(facultyLink)
        soup2 = BeautifulSoup(re.content, 'html.parser')
        
        facultyId = soup2.find('script', id='burst-js-extra').text.split(',')[1].split(':')[-1].strip(' "')
        
        isSupervisor = False
        isSupervisor = soup2.find('span', class_='gdlr-core-title-item-caption').text.strip().lower().split('\n')[-1] == 'hec approved phd supervisor'
        
        #########################
        facultyEducation = None  # Initialize to None

        # Case 1
        span = soup2.find('h3', class_='gdlr-core-title-item-title gdlr-core-skin-title', string='Education')
        if span:
            span2 = span.find_next('span', class_='gdlr-core-title-item-caption gdlr-core-info-font gdlr-core-skin-caption')
            if span2:
                facultyEducation = span2.text.split(',')[0]

        # Case 2
        if facultyEducation is None:
            education_div = soup2.find('h3', class_='gdlr-core-title-item-title gdlr-core-skin-title', string='Education')
            if education_div:
                ul = education_div.find_next('ul')
                if ul:
                    li = ul.find_next('li')
                    if li and li.text:
                        facultyEducation = li.text.split(',')[0].split('from')[0]
                        
        # Case 3                 
        if facultyEducation is None or 'Facebook' in facultyEducation:
            p_class = soup2.find('h3', class_='gdlr-core-title-item-title gdlr-core-skin-title', string='Education')
            if p_class:
                class2 = p_class.find_next('p')
                if class2:
                    facultyEducation = class2.text.strip().split(',')[0]
         ###########################           
                    
        facultyIMG = None  # Initialize to None
        img_div = soup2.find('div', class_='gdlr-core-image-item-wrap')
        if img_div:
            img_a = img_div.find('a', class_='gdlr-core-lightgallery')
            if img_a:
                facultyIMG = img_a['href']
                
        ################################
        
        # Append to each list
        
        try:
            id_.append(int(facultyId))
        except:
            id_.append(-1)
        
        name.append(str(facultyName))
        designation.append(str(facultyDesignation))
        phdSupervisor.append(bool(isSupervisor))
        email.append(facultyEmail)
        dept.append(str(facultyDepartment))

        try:
            ext.append(int(facultyExtension))
        except:
            ext.append(-1)

        img.append(facultyIMG)
        highesteducation.append(str(facultyEducation))

In [32]:
df = pd.DataFrame({'ID':id_,
                   'Name':name,
                   'Designation':designation,
                   'HEC Approved PHD Supervisor':phdSupervisor,
                   'Highest Education':highesteducation,
                   'Email':email,
                   'Department':dept,
                   'Extension':ext,
                   'ImageURL':img})

In [33]:
df

,ID,Name,Designation,HEC Approved PHD Supervisor,Highest Education,Email,Department,Extension,ImageURL
0,7415,Dr. Syed Muhammad Fahad Riaz,Assistant Professor & Incharge (SH),True,PhD (Mathematics),fahad.riaz@nu.edu.pk,Department of Sciences & Humanities,267,https://khi.nu.edu.pk/wp-content/uploads/2023/...
1,7421,Dr. Khusro Mian,Associate Professor,False,PhD (Astro Phy.),khusro.mian@nu.edu.pk,Department of Sciences & Humanities,295,https://khi.nu.edu.pk/wp-content/uploads/2023/...
2,7429,Dr. Muhammad Shahzad Shaikh,Assistant Professor,False,PhD (Islamic Stud.),shahzad.sheikh@nu.edu.pk,Department of Sciences & Humanities,248,https://khi.nu.edu.pk/wp-content/uploads/2023/...
3,7433,Dr. Nazish Kanwal,Assistant Professor,False,PhD (M),nazish.kanwal@nu.edu.pk,Department of Sciences & Humanities,303,https://khi.nu.edu.pk/wp-content/uploads/2023/...
4,7437,Dr. Shahnawaz Muhammad Khan,Assistant Professor,False,PhD (Int. Rel.),-,Department of Sciences & Humanities,305,https://khi.nu.edu.pk/wp-content/uploads/2023/...
...,...,...,...,...,...,...,...,...,...
128,7076,Ms. Syeda Rubab,Lecturer,False,None,syeda.rubab@nu.edu.pk,Department of Software Engineering,279,https://khi.nu.edu.pk/wp-content/uploads/2023/...
129,7077,Mr. Minhal Raza,Lecturer,False,None,minhal.raza@nu.edu.pk,Department of Software Engineering,190,https://khi.nu.edu.pk/wp-content/uploads/2023/...
130,7081,Mr. Muhammad Nadeem,Instructor,False,None,muhammad.nadeem@nu.edu.pk,Department of Software Engineering,293,https://khi.nu.edu.pk/wp-content/uploads/2023/...
131,7082,Ms. Ayesha Ali,Instructor,False,None,ayesha.ali@nu.edu.pk,Department of Software Engineering,293,https://khi.nu.edu.pk/wp-content/uploads/2023/...


In [ ]:
df.to_csv('khi.csv')

In [ ]:
df.isnull().sum()

In [ ]:
df[df.ImageURL.isnull()]

In [ ]:
df[df['Highest Education'] == 'Education']

# Test for one faculty member

In [ ]:
response = requests.get(department_links[0])
response

In [ ]:
soup = BeautifulSoup(response.content, 'html.parser')
faculty = soup.findAll('div', class_='gdlr-core-personnel-list gdlr-core-outer-frame-element clearfix')[2]
#print(faculty.prettify())

In [ ]:
facultyName = faculty.find('h3', class_='gdlr-core-personnel-list-title').text.strip().split(',')[0]
facultyName

In [ ]:
facultyDesignation = faculty.find('div', class_='gdlr-core-personnel-list-position gdlr-core-info-font gdlr-core-skin-caption').text.strip()
facultyDesignation

In [ ]:
facultyEmail = faculty.find('div', class_='kingster-personnel-info-list kingster-type-email').text.strip()
facultyEmail

In [ ]:
facultyDepartment = faculty.find_previous('h1', class_='kingster-page-title').text.strip()
facultyDepartment

In [ ]:
facultyExtension = faculty.find('div', class_='kingster-personnel-info-list kingster-type-phone').text.strip()
facultyExtension

In [ ]:
facultyLink = faculty.find('h3', class_='gdlr-core-personnel-list-title').find_next('a')['href']
facultyLink

In [14]:
re = requests.get('https://khi.nu.edu.pk/personnel/prof-dr-zulfiqar-ali-memon-phd/')
soup2 = BeautifulSoup(re.content, 'html.parser')

In [24]:
facultyId = soup2.find('script', id='burst-js-extra').text.split(',')[1].split(':')[-1].strip(' "')
facultyId

'6281'

In [ ]:
facultyIMG = None  # Initialize to None
img_div = soup2.find('div', class_='gdlr-core-image-item-wrap')
if img_div:
    img_a = img_div.find('a', class_='gdlr-core-lightgallery')
    if img_a:
        facultyIMG = img_a['href']
facultyIMG

In [ ]:
isSupervisor = False
isSupervisor = soup2.find('span', class_='gdlr-core-title-item-caption').text.strip().lower().split('\n')[-1] == 'hec approved phd supervisor'
isSupervisor

In [ ]:
# facultyEducation = None  # Initialize to None
# education_div = soup2.find('h3', class_='gdlr-core-title-item-title gdlr-core-skin-title', string='Education')
# if education_div:
#     ul = education_div.find_next('ul')
#     if ul:
#         li = ul.find_next('li')
#         if li and li.text:
#             facultyEducation = li.text.split(',')[0]

In [ ]:
facultyEducation = None  # Initialize to None

span = soup2.find('h3', class_='gdlr-core-title-item-title gdlr-core-skin-title', string='Education')
if span:
    span2 = span.find_next('span', class_='gdlr-core-title-item-caption gdlr-core-info-font gdlr-core-skin-caption')
    if span2:
        facultyEducation = span2.text.split(',')[0]

if facultyEducation is None:
    education_div = soup2.find('h3', class_='gdlr-core-title-item-title gdlr-core-skin-title', string='Education')
    if education_div:
        ul = education_div.find_next('ul')
        if ul:
            li = ul.find_next('li')
            if li and li.text:
                facultyEducation = li.text.split(',')[0]

if facultyEducation is None or 'Facebook' in facultyEducation:
    p_class = soup2.find('h3', class_='gdlr-core-title-item-title gdlr-core-skin-title', string='Education')
    if p_class:
        class2 = p_class.find_next('p')
        if class2:
            facultyEducation = class2.text.split(',')[0]
            
facultyEducation